# DATA PROCESSING

First, we import all data. Data was provided by [Football-Data.co.uk](https://www.football-data.co.uk/data.php)

In [1]:
leagues = ["eng", "spa", "ita", "ger", "fra"]
years = ["2122", "2021", "1920"]
file_str = []

for league in leagues:
    for year in years:
        file_str.append(league + year + ".csv")

file_str[:2]

['eng2122.csv', 'eng2021.csv']

Statistics, that will come into account (and we want to keep):
- team
- goals
- referee
- shots
- shots on target
- corners
- booking points (10 for yellow and 25 for red card)
- bet365 odds for:
    - final result
    - over 2.5 total goals

In [2]:
import pandas as pd

spa2122 = pd.read_csv('data/spain/spa2122.csv')
spa2021 = pd.read_csv('data/spain/spa2021.csv')
spa1920 = pd.read_csv('data/spain/spa1920.csv')
spa1819 = pd.read_csv('data/spain/spa1819.csv')
spa1718 = pd.read_csv('data/spain/spa1718.csv')
spa1617 = pd.read_csv('data/spain/spa1617.csv')
spa1516 = pd.read_csv('data/spain/spa1516.csv')
spa1415 = pd.read_csv('data/spain/spa1415.csv')
spa1314 = pd.read_csv('data/spain/spa1314.csv')

In [3]:
spa2122 = spa2122[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]
spa2021 = spa2021[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]
spa1920 = spa1920[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]
spa1819 = spa1819[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]
spa1718 = spa1718[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]
spa1617 = spa1617[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]
spa1516 = spa1516[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]
spa1415 = spa1415[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]
spa1314 = spa1314[["Date", "HomeTeam","AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HY", "AY", "B365H", "B365D", "B365A"]]

In [4]:
table = pd.concat(list(reversed([spa2122, spa2021, spa1920, spa1819, spa1718, spa1617, spa1516, spa1415, spa1314])))

table

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HC,AC,HY,AY,B365H,B365D,B365A
0,17/08/13,Sociedad,Getafe,2,0,16,15,6,2,6,5,1,1,1.73,3.60,4.75
1,17/08/13,Valencia,Malaga,1,0,9,11,1,2,9,6,3,5,1.53,4.00,6.00
2,17/08/13,Valladolid,Ath Bilbao,1,2,8,13,2,3,5,5,1,0,2.50,3.30,2.80
3,18/08/13,Barcelona,Levante,7,0,22,4,13,1,9,3,1,3,1.08,10.00,26.00
4,18/08/13,Osasuna,Granada,1,2,14,13,5,4,7,6,1,4,2.00,3.30,3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,22/05/2022,Granada,Espanol,0,0,15,10,5,5,7,3,1,0,1.44,4.33,7.50
376,22/05/2022,Osasuna,Mallorca,0,2,11,14,1,6,5,7,2,2,3.30,3.40,2.20
377,22/05/2022,Barcelona,Villarreal,0,2,14,4,3,3,12,1,3,1,2.10,4.00,3.00
378,22/05/2022,Sevilla,Ath Bilbao,1,0,13,9,3,1,5,0,3,3,2.50,3.30,2.87


Now we normalize the table, so each row represents a match for each team.

In [5]:
from re import A

df = pd.DataFrame(columns=['index', 'date', 'home_team', 'away_team', 'goals_home', 'goals_away',
                                'result', 'shots_home', 'shots_away', 'shots_on_target_home', 'shots_on_target_away',
                                'corners_home', 'corners_away', 'yellow_cards_home', 'yellow_cards_away', 'odds_home',
                                'odds_draw', 'odds_away'])

def result_home(row):
    if (row.FTHG > row.FTAG): return 'home win'
    elif (row.FTHG == row.FTAG): return 'draw'
    else: return 'away win'

index = 0
for id,row in table.iterrows():
    
    dict_home = {'index': index,
                 'date': [row.Date],
                 'home_team': [row.HomeTeam],
                 'away_team': [row.AwayTeam], 
                 'goals_home': [row.FTHG], 
                 'goals_away': [row.FTAG],
                 'result': [result_home(row)], 
                 'shots_home': [row.HS], 
                 'shots_away': [row.AS], 
                 'shots_on_target_home': [row.HST], 
                 'shots_on_target_away': [row.AST],
                 'corners_home': [row.HC], 
                 'corners_away': [row.AC], 
                 'yellow_cards_home': [row.HY], 
                 'yellow_cards_away': [row.AY], 
                 'odds_home': [row.B365H],
                 'odds_draw': [row.B365D], 
                 'odds_away': [row.B365A]}

    index += 1
    temp_df_home = pd.DataFrame.from_dict(dict_home)

    df = pd.concat([df, temp_df_home])

df = df.set_index('index')
df

,date,home_team,away_team,goals_home,goals_away,result,shots_home,shots_away,shots_on_target_home,shots_on_target_away,corners_home,corners_away,yellow_cards_home,yellow_cards_away,odds_home,odds_draw,odds_away
index,,,,,,,,,,,,,,,,,
0,17/08/13,Sociedad,Getafe,2,0,home win,16,15,6,2,6,5,1,1,1.73,3.60,4.75
1,17/08/13,Valencia,Malaga,1,0,home win,9,11,1,2,9,6,3,5,1.53,4.00,6.00
2,17/08/13,Valladolid,Ath Bilbao,1,2,away win,8,13,2,3,5,5,1,0,2.50,3.30,2.80
3,18/08/13,Barcelona,Levante,7,0,home win,22,4,13,1,9,3,1,3,1.08,10.00,26.00
4,18/08/13,Osasuna,Granada,1,2,away win,14,13,5,4,7,6,1,4,2.00,3.30,3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,22/05/2022,Granada,Espanol,0,0,draw,15,10,5,5,7,3,1,0,1.44,4.33,7.50
3416,22/05/2022,Osasuna,Mallorca,0,2,away win,11,14,1,6,5,7,2,2,3.30,3.40,2.20
3417,22/05/2022,Barcelona,Villarreal,0,2,away win,14,4,3,3,12,1,3,1,2.10,4.00,3.00


## Data that we will use for prediction:
We will try to predict, based on:
- last 3 games
- last 5 games

Statistics, on which we will predict result probabilites (all averages):
- points 
- goals scored
- goals conceded
- shots for
- shots against
- corners difference
- all of the opponenets statistics
- home team

In [6]:
# index for easier calc
game_index = {}

for id, value in df.iterrows():
    try:
        game_index[value['home_team']].append(('home', id))
    except:
        game_index[value['home_team']] = [('home', id)]

    try:
        game_index[value['away_team']].append(('away', id))
    except:
        game_index[value['away_team']] = [('away', id)]

game_index

{'Sociedad': [('home', 0),
  ('away', 12),
  ('home', 28),
  ('away', 32),
  ('home', 42),
  ('away', 51),
  ('home', 63),
  ('away', 75),
  ('away', 83),
  ('home', 97),
  ('away', 105),
  ('home', 113),
  ('away', 125),
  ('home', 134),
  ('away', 144),
  ('home', 158),
  ('away', 163),
  ('home', 177),
  ('away', 189),
  ('away', 196),
  ('home', 209),
  ('away', 216),
  ('home', 227),
  ('away', 239),
  ('home', 244),
  ('away', 257),
  ('home', 269),
  ('home', 278),
  ('away', 289),
  ('home', 299),
  ('away', 304),
  ('home', 313),
  ('away', 321),
  ('home', 333),
  ('away', 340),
  ('home', 358),
  ('away', 362),
  ('home', 377),
  ('away', 386),
  ('home', 398),
  ('away', 402),
  ('home', 417),
  ('away', 426),
  ('home', 439),
  ('away', 447),
  ('home', 459),
  ('away', 462),
  ('home', 474),
  ('home', 488),
  ('away', 494),
  ('home', 500),
  ('away', 517),
  ('home', 528),
  ('away', 533),
  ('home', 546),
  ('away', 557),
  ('home', 561),
  ('home', 572),
  ('away', 58

In [7]:
n = 5

temp_stats = {}

for i in range(50, len(df)):
    temp_stats[i] = {}


for key, lst in game_index.items():
    for id, value in enumerate(lst):
        if id < n:
            pass
        else:
            # first get game indexes
            games_accounted = game_index[key][(id - n):id]

            game_type, index = value
            game = df.iloc[index]

            # already columns
            result = game['result']
            odds_home = game['odds_home']
            odds_draw = game['odds_draw']
            odds_away = game['odds_away']
            
            points = 0
            goals_scored = 0
            goals_conceded = 0
            shots_given = 0
            shots_conceded = 0
            corners_difference = 0

            for (ref_game_type, ref_index) in games_accounted:
                ref_game = df.iloc[ref_index]

                if ref_game_type == 'home':

                    if ref_game['result'] == 'home win':
                        points += 3
                    elif ref_game['result'] == 'draw':
                        points += 1
                    else:
                        pass

                    goals_scored += ref_game['goals_home']
                    goals_conceded += ref_game['goals_away']

                    shots_given += ref_game['shots_home']
                    shots_conceded += ref_game['shots_away']

                    corners_difference += ref_game['corners_home'] - ref_game['corners_away'] 

                else:
                    if ref_game['result'] == 'away win':
                        points += 3
                    elif ref_game['result'] == 'draw':
                        points += 1
                    else:
                        pass

                    goals_scored += ref_game['goals_away']
                    goals_conceded += ref_game['goals_home']

                    shots_given += ref_game['shots_away']
                    shots_conceded += ref_game['shots_home']

                    corners_difference += ref_game['corners_away'] - ref_game['corners_home']

            if game_type == 'home':
                temp_stats[index]['result'] = result
                temp_stats[index]['odds_home'] = odds_home
                temp_stats[index]['odds_draw'] = odds_draw
                temp_stats[index]['odds_away'] = odds_away
                temp_stats[index]['points_home'] = points
                temp_stats[index]['goals_scored_home'] = goals_scored
                temp_stats[index]['goals_conceded_home'] = goals_conceded
                temp_stats[index]['shots_given_home'] = shots_given
                temp_stats[index]['shots_conceded_home'] = shots_conceded
                temp_stats[index]['corners_difference_home'] = corners_difference

            else:
                temp_stats[index]['result'] = result
                temp_stats[index]['odds_home'] = odds_home
                temp_stats[index]['odds_draw'] = odds_draw
                temp_stats[index]['odds_away'] = odds_away
                temp_stats[index]['points_away'] = points
                temp_stats[index]['goals_scored_away'] = goals_scored
                temp_stats[index]['goals_conceded_away'] = goals_conceded
                temp_stats[index]['shots_given_away'] = shots_given
                temp_stats[index]['shots_conceded_away'] = shots_conceded
                temp_stats[index]['corners_difference_away'] = corners_difference
                    
    print(f"Done: {key}")



Done: Sociedad
Done: Getafe
Done: Valencia
Done: Malaga
Done: Valladolid
Done: Ath Bilbao
Done: Barcelona
Done: Levante
Done: Osasuna
Done: Granada
Done: Real Madrid
Done: Betis
Done: Sevilla
Done: Ath Madrid
Done: Almeria
Done: Villarreal
Done: Celta
Done: Espanol
Done: Vallecano
Done: Elche
Done: La Coruna
Done: Eibar
Done: Cordoba
Done: Las Palmas
Done: Sp Gijon
Done: Alaves
Done: Leganes
Done: Girona
Done: Huesca
Done: Mallorca
Done: Cadiz


In [8]:
df.iloc[1560]

date                     19/09/17
home_team               Barcelona
away_team                   Eibar
goals_home                      6
goals_away                      1
result                   home win
shots_home                     11
shots_away                     13
shots_on_target_home           10
shots_on_target_away            4
corners_home                    7
corners_away                    7
yellow_cards_home               0
yellow_cards_away               2
odds_home                    1.11
odds_draw                    10.0
odds_away                    15.0
Name: 1560, dtype: object

In [16]:
temp_stats[381]

{'result': 'home win',
 'odds_home': 1.95,
 'odds_draw': 3.2,
 'odds_away': 4.2,
 'points_home': 4,
 'goals_scored_home': 2,
 'goals_conceded_home': 10,
 'shots_given_home': 61,
 'shots_conceded_home': 65,
 'corners_difference_home': -4}

In [10]:
df.to_csv('data/spain_combined.csv')

In [21]:
all_statistics = pd.DataFrame(columns=[
                                'result', 'odds_home', 'odds_draw', 'odds_away',
                                'points_home', 'points_away',
                                'goals_scored_home', 'goals_scored_away',
                                'goals_conceded_home', 'goals_conceded_away',
                                'shots_given_home', 'shots_given_away',
                                'shots_conceded_home', 'shots_conceded_away',
                                'corners_difference_home', 'corners_difference_away'
                                ])


for key, value in temp_stats.items():
    try:
        temp = pd.DataFrame.from_dict({
                'index': [int(key)],
                'result': [value['result']],
                'odds_home': [value['odds_home']],
                'odds_draw': [value['odds_draw']],
                'odds_away': [value['odds_away']],
                'points_home': [value['points_home']],
                'points_away': [value['points_away']],
                'goals_scored_home': [value['goals_scored_home']],
                'goals_scored_away': [value['goals_scored_away']],
                'goals_conceded_home': [value['goals_conceded_home']],
                'goals_conceded_away': [value['goals_conceded_away']],
                'shots_given_home': [value['shots_given_home']],
                'shots_given_away': [value['shots_given_away']],
                'shots_conceded_home': [value['shots_conceded_home']],
                'shots_conceded_away': [value['shots_conceded_away']],
                'corners_difference_home': [value['corners_difference_home']],
                'corners_difference_away': [value['corners_difference_away']]})
    except KeyError:
        # when a well established team, like Sociedad plays a incoming lower division team, we do not possess their data: 
        # therefore: skip
        continue

    all_statistics = pd.concat([all_statistics, temp])

all_statistics = all_statistics.set_index(['index'])

In [23]:
print(all_statistics)
all_statistics.to_pickle('../simulation/input_data.pickle')
all_statistics.to_csv('../simulation/input_data.csv')

          result  odds_home  odds_draw  odds_away points_home points_away  \
index                                                                       
50.0    home win       1.20       6.00      15.00          15           3   
51.0    home win       1.13       8.00      19.00          15           6   
52.0        draw       2.10       3.30       3.50           6           4   
53.0    home win       1.67       3.75       5.00           5           3   
54.0    away win      10.00       5.50       1.29           3          13   
...          ...        ...        ...        ...         ...         ...   
3415.0      draw       1.44       4.33       7.50           8           2   
3416.0  away win       3.30       3.40       2.20           3           7   
3417.0  away win       2.10       4.00       3.00          10           7   
3418.0  home win       2.50       3.30       2.87           7          10   
3419.0  away win       2.62       3.30       2.70           7           8   